<a href="https://colab.research.google.com/github/AravindDisha/CAA-Twitter-Analysis/blob/main/MetricCalculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install botometer

In [ ]:
!pip install twint                                  

In [ ]:
!pip install googletrans

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#mount on other drive account for more space

In [ ]:
import IPython
import ast
import csv
import datetime
import json
import math
import matplotlib.pyplot as plt
import statistics 
from statistics import mode 
import networkx as nx

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

import numpy as np
import os.path
import pandas as pd
import pickle
import re
import requests
import string
import sys
import time
import twint
from IPython.display import HTML,display
from datetime import timedelta
from googletrans import Translator

from keras.callbacks import EarlyStopping
from keras.layers import Dense, Activation
from keras.layers import Embedding
from keras.layers import Flatten, Conv1D, SpatialDropout1D, MaxPooling1D,AveragePooling1D, Bidirectional, merge, concatenate, Input, Dropout, LSTM
from keras.models import Sequential, Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

from operator import itemgetter
from os import listdir
from os import path
from os.path import isfile, join
from pprint import pprint

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.externals.six import StringIO
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate as cross_validation, ShuffleSplit, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

# import warnings
translator = Translator()
# warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
import tweepy as tw
#Add your access codes here if you get yours. Only 320 requests are allowed per day I think
consumer_key= ~
consumer_secret= ~
access_token= ~
access_token_secret= ~
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
# For Representation
class Tweet(object):
    def __init__(self, s, embed_str=False):
        if not embed_str:
            # Use Twitter's oEmbed API
            # https://dev.twitter.com/web/embedded-tweets
            api = 'https://publish.twitter.com/oembed?url={}'.format(s)
            response = requests.get(api)
            self.text = response.json()["html"]
        else:
            self.text = s

    def _repr_html_(self):
        return self.text

In [ ]:
col_names = ['id','conversation_id',	'created_at',	'date',	'time',	'timezone',	'user_id',	'username',	'name',	'place',	'tweet',	'mentions',	'urls',	'photos',	'replies_count',	'retweets_count',	'likes_count',	'hashtags',	'cashtags',	'link',	'retweet',	'quote_url',	'video',	'near',	'geo',	'source',	'user_rt_id',	'user_rt',	'retweet_id',	'reply_to',	'retweet_date',	'translate',	'trans_src',	'trans_dest']

# Load Stance Models

In [ ]:
# All models for stance detection
filenames = ["/content/drive/My Drive/stance_data/models/Support_Vector_Machine_1.pkl","/content/drive/My Drive/stance_data/models/Random_Forest_Classifier_1.pkl","/content/drive/My Drive/stance_data/models/Gradient_Boosting_Classifier_1.pkl","/content/drive/My Drive/stance_data/models/Logistic_Regression_1.pkl","/content/drive/My Drive/stance_data/models/Neural_Network_1.pkl","/content/drive/My Drive/stance_data/models/Gaussian_NB_1.pkl","/content/drive/My Drive/stance_data/models/K_Neighbors_Classifier_1.pkl","/content/drive/My Drive/stance_data/models/Decision_Tree_Classifier_1.pkl","/content/drive/My Drive/stance_data/models/Support_Vector_Machine_2.pkl","/content/drive/My Drive/stance_data/models/Random_Forest_Classifier_2.pkl","/content/drive/My Drive/stance_data/models/Gradient_Boosting_Classifier_2.pkl","/content/drive/My Drive/stance_data/models/Logistic_Regression_2.pkl","/content/drive/My Drive/stance_data/models/Neural_Network_2.pkl","/content/drive/My Drive/stance_data/models/Gaussian_NB_2.pkl","/content/drive/My Drive/stance_data/models/K_Neighbors_Classifier_2.pkl","/content/drive/My Drive/stance_data/models/Decision_Tree_Classifier_2.pkl","/content/drive/My Drive/stance_data/models/lstm1_3.pkl","/content/drive/My Drive/stance_data/models/lstm2_3.pkl","/content/drive/My Drive/stance_data/models/lstm1_4.pkl","/content/drive/My Drive/stance_data/models/lstm2_4.pkl"]

In [ ]:
# models being used currently
print('\n'.join(filenames[2:5]))
print(filenames[17])

/content/drive/My Drive/stance_data/models/Gradient_Boosting_Classifier_1.pkl
/content/drive/My Drive/stance_data/models/Logistic_Regression_1.pkl
/content/drive/My Drive/stance_data/models/Neural_Network_1.pkl
/content/drive/My Drive/stance_data/models/lstm2_3.pkl


In [ ]:
# Load from file
models1 = {}
for pkl_filename in filenames[:8]:
  with open(pkl_filename, 'rb') as file:
      pickle_model = pickle.load(file)
      # print(pickle_model)
      models1[pkl_filename] = pickle_model

models2 = {}
for pkl_filename in filenames[8:16]:
  with open(pkl_filename, 'rb') as file:
      pickle_model = pickle.load(file)
      models2[pkl_filename] = pickle_model

models3 = {}
for pkl_filename in filenames[16:18]:
  with open(pkl_filename, 'rb') as file:
      pickle_model = pickle.load(file)
      models3[pkl_filename] = pickle_model

models4 = {}
for pkl_filename in filenames[18:]:
  with open(pkl_filename, 'rb') as file:
      pickle_model = pickle.load(file)
      models4[pkl_filename] = pickle_model

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
# Functions from the stance models to help predict the stance
def labelStance(labelDict, data):
	for key, val in labelDict.items():
		data.loc[data["Stance"] == val, "Stance"] = int(key)
	return data

def readGlobalVecData(glove_word_vec_file):
	file = open(glove_word_vec_file, encoding="utf8")
	rawData = file.readlines()
	glove_word_vec_dict = {}
	for line in rawData:
		line = line.strip().split()
		tag = line[0]
		vec = line[1:]
		glove_word_vec_dict[tag] = np.array(vec, dtype=float)
	return glove_word_vec_dict

def getWordVector(word, glove_word_vec_dict):
	if word in glove_word_vec_dict:
		return glove_word_vec_dict[word]
	return np.zeros_like(glove_word_vec_dict["dummy"])

def sumVectors(finalList, glove_word_vec_dict):
	numNonZero = 0
	vector = np.zeros_like(glove_word_vec_dict["dummy"])
	for word in finalList:
		vect = getWordVector(word,glove_word_vec_dict)
		if vect.sum() != 0:
			vector += vect
			numNonZero += 1
	if numNonZero:
		vector = vector/numNonZero
	return vector

def simplify(word):
	dump = ''
	temp = []
	listOfWords = list(filter(None,re.split("([A-Z][^A-Z]*)",word)))
	if len(listOfWords) == len(word):
		return word.lower()
	for i in range(len(listOfWords)):
		listOfWords[i] = listOfWords[i].lower()
		if len(listOfWords[i]) == 1:
			dump = dump + listOfWords[i]
			if dump in words.words() and len(dump) > 2:
				temp.append(dump)
				dump = ''
		else:
			temp.append(listOfWords[i])
	return temp

def glove_test(glove_word_vec_dict, trainTweets):
    def createTokens(data,glove_word_vec_dict):
        # listOfTweets = []
        # listOfStances = []
        tweetVector = []
        for text in data:
            # Create a sentence using target and the tweet. Word vector will be formed from this.
            example_sentence = 'CAA' +' '+ text
            # Remove punctuation
            final_sentence = example_sentence.translate(string.punctuation)
            wordList = word_tokenize(final_sentence)
            finalList = []
            s = ' '.join([i for i in wordList if i.isalpha()])
            # create tokens from the string and stem them
            wordList = word_tokenize(s)
            wordList = [w.lower() for w in wordList]
            stop_words = set(stopwords.words('english'))
            wordList = [w for w in wordList if not w in stop_words]
            for word in wordList:
                #to break any combined word into its components for eg, hashtags
                finalList += simplify(word)
            final_sentence = ' '.join(finalList)
            # listOfTweets.append(final_sentence)
            # listOfStances.append(row["Stance"])
            tweetVector.append(sumVectors(finalList,glove_word_vec_dict))
        return tweetVector
    # Remove punctuation from and tokenize the tweets
    testTweetVector = createTokens(trainTweets, glove_word_vec_dict)
    return testTweetVector

gloveFile = "/content/drive/My Drive/stance_data/glove/glove.twitter.27B.200d.txt"

print("\nLoading Glove data in progress...")
glove_word_vec_dict = readGlobalVecData(gloveFile)
print("\nLoading Glove data is done...")


Loading Glove data in progress...

Loading Glove data is done...


In [ ]:
# Functions from the stance models to help predict the stance 
file = "/content/drive/My Drive/stance_data/demon/csv_files/demoapril.csv"
df = pd.read_csv(file,encoding='utf-8')
df.info()
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = str(text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

df['text'] = df.text + ' ' + 'Demonitization'
df['text'] = df['text'].apply(clean_text)
df['text'] = df['text'].str.replace('\d+', '')

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 100
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
tokenizer1 = tokenizer

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2817 entries, 0 to 2816
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      2817 non-null   object
 1   text    2814 non-null   object
 2   Stance  2811 non-null   object
dtypes: object(3)
memory usage: 66.1+ KB
Found 6540 unique tokens.


In [ ]:
# training = "/content/drive/My Drive/stance_data/semval/training.txt"
# df = pd.read_csv(training, sep='\t',header=0,encoding='utf-8')
# df.info()
# df = df.reset_index(drop=True)
# REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
# BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
# STOPWORDS = set(stopwords.words('english'))

# def clean_text(text):
#     text = text.lower() # lowercase text
#     text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
#     text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#     text = text.replace('x', '')
#     text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
#     return text

# df['Tweet'] = df.Tweet + ' ' + df.Target
# df['Tweet'] = df['Tweet'].apply(clean_text)
# df['Tweet'] = df['Tweet'].str.replace('\d+', '')

# # The maximum number of words to be used. (most frequent)
# MAX_NB_WORDS = 50000
# # Max number of words in each complaint.
# MAX_SEQUENCE_LENGTH = 100
# # This is fixed.
# EMBEDDING_DIM = 100
# tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
# tokenizer.fit_on_texts(df['Tweet'].values)
# word_index = tokenizer.word_index
# print('Found %s unique tokens.' % len(word_index))
# tokenizer2 = tokenizer

# Find most retweeted tweets

In [ ]:
# Code to find top 1-1500 retweeted people
# Kind of binary search between min and max of tweets present
def FindMostRetweeted(hashtag,max_taken = 4300000,min_taken=0):
  
  if(max_taken <= min_taken+2):
    print('revert to previous answer')
    return None
  p_val = ""
  val = int((max_taken+min_taken)/2)
  c = twint.Config()
  c.Limit = 2000
  c.Search = hashtag
  c.Min_retweets = val
  c.Store_csv = True
  csv_name = "temp_out"+str(val)+".csv"
  c.Output = csv_name
  twint.run.Search(c)
  
  t_df = None
  if(path.exists(csv_name)):
    p_val += '\ntweets found for '+str(val)
    t_df = pd.read_csv(csv_name, delimiter=',',names = col_names,skiprows=1)
    IPython.display.clear_output()
  
  t1_df = None
  if((type(t_df)==type(None)) or (t_df.id.count() < 1)):
    # no tweets found 
    p_val += '\nneed more people : less rt '+str(val)
    t1_df = FindMostRetweeted(hashtag,val,min_taken)
  elif(t_df.id.count() > 1500):
    # too many found
    val1 = sorted(list(t_df.retweets_count))[-15]
    p_val+= '\nneed less people : more rt '+str(val1)
    t1_df = FindMostRetweeted(hashtag,max_taken,val1)
  else:
    # correct number found
    p_val+= '\nFound at '+str(val)+', tweets = '+str(t_df.id.count())
  
  try:
    os.remove(csv_name)
  except:
    p_val += "\nno tweets found for "+str(val)

  if(not (type(t1_df) == type(None))):
    print(p_val)
    return t1_df
  print(p_val)
  return t_df

In [ ]:
# most retweeted tweets found
df_most = FindMostRetweeted('caa OR nrc OR npr')

# Get all Retweets, Original tweets and Replies for a user by user_id

In [ ]:
# Use twint search to get the retweets by user which have that hashtag
def getAllRetweets(usr_id,dir_path,hashtag=''):
  c = twint.Config()
  c.User_id = usr_id
  c.Replies = False
  if(hashtag!=''):
    c.Search= hashtag
  c.Retweets = True
  c.Native_retweets = True
  c.Store_csv = True
  csv_name = dir_path+"rts.csv"
  c.Output = csv_name
  twint.run.Search(c)

  df = None
  if(path.exists(csv_name)):
    df = pd.read_csv(csv_name, delimiter=',',names = col_names,skiprows=1)
    IPython.display.clear_output()
  return df

# Use twint search to get the original tweets and replies by user which have that hashtag
def getAllOriginalAndReplies(usr_id,dir_path,hashtag=''):
  c = twint.Config()
  c.User_id = usr_id
  if(hashtag!=''):
    c.Search= hashtag
  c.Replies = False
  c.Retweets = False
  c.Native_retweets = False
  c.Store_csv = True
  csv_name = dir_path+"og_rep.csv"
  c.Output = csv_name
  twint.run.Search(c)

  df1 = None
  df2 = None
  if(path.exists(csv_name)):
    df = pd.read_csv(csv_name, delimiter=',',names = col_names,skiprows=1)
    IPython.display.clear_output()
    df1 = df[df['id'] == df['conversation_id']] #Original Tweets
    df1 = df1.reset_index(drop=True)
    df2 = df[df['id'] != df['conversation_id']] #Reply Tweets
    df2 = df2.reset_index(drop=True)
  return df1,df2

# Calls all the above functions to send the set of dataf
def getTweets(usr_id,hashtag=''):
  dir_path = '/content/drive/My Drive/TweetData/tweets_'+str(usr_id)
  # cached
  if(usr_id in all_dfs.keys()):
    # dfs = all_dfs[usr_id]['dataframes']
    # return dfs[0],dfs[1],dfs[2],dir_path
    return None,None,None,"already there"
  try:
    os.mkdir(dir_path)
    df1 = getAllRetweets(usr_id,dir_path+'/',hashtag)
    df2,df3 = getAllOriginalAndReplies(usr_id,dir_path+'/',hashtag)
    try:
      os.rmdir(dir_path)
    except:
      # print('tweets present')
      ;
    return df1,df2,df3,dir_path
  except:
    # print("already there")
    return None,None,None,"already there"


# Get the stance predicted by model

In [ ]:
RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
tag_pattern = re.compile(r"[A-Z][a-z]+|\d+|[A-Z]+(?![a-z])")

def hashtag_to_sent(tag):
    return " ".join(tag_pattern.findall(tag))

def convert_hashtags(twt):
  word_list = twt.split(' ')
  twt = ""
  for word in word_list:
    if(word.startswith('#')):
      twt+=' '+hashtag_to_sent(word[1:])
    else:
      twt+=' '+word
  return twt

def translate_tweet(twts):
  tw_list = [convert_hashtags(RE_EMOJI.sub(r'', twt)) for twt in twts]
  try:
    # trans_twt = translator.translate(convert_hashtags(twt),dest='en').text
    translations = translator.translate(tw_list, dest='en')
    return[t.text for t in translations]
  except:
    trans_twt = [twt.encode('utf-8').decode() for twt in twts] 
    # print(trans_twt)
    print('translator failed')
    return trans_twt

def getPredictions(tweet_list):
  tweet_list = translate_tweet(tweet_list)
  Ltest = glove_test(glove_word_vec_dict,tweet_list)
  results = []
  fin = {'F':0,'A':0,'N':0}
  labels_stance = {0:"A", 1:"F", 2:"N"}

  for classifier in filenames[2:5]:
    res_arr = models1[classifier].predict(Ltest)
    temp = {'F':0,'A':0,'N':0,"":0}
    for i in res_arr:
      temp[labels_stance[i%3]]+=1
      fin[labels_stance[i%3]]+=1
    tot = sum(temp.values())
    temp = {key:temp[key]/tot for key in temp.keys()}
    if(temp['A'] > temp['F']):
      temp['res'] = 'A'
    elif(temp['A'] < temp['F']):
      temp['res'] = 'F'
    else:
      temp['res'] = 'N'
    results.append(temp['res'])
  
  # for classifier in filenames[8:16]:
  #   res_arr = models2[classifier].predict(Ltest)
  #   temp = {'F':0,'A':0,'N':0}
  #   for i in res_arr:
  #     temp[labels_stance[i%3]]+=1
  #     # fin[labels_stance[i]]+=1
  #   tot = sum(temp.values())
  #   temp = {key:temp[key]/tot for key in temp.keys()}
  #   if(temp['A'] > temp['F']):
  #     temp['res'] = 'A'
  #   elif(temp['A'] < temp['F']):
  #     temp['res'] = 'F'
  #   else:
  #     temp['res'] = 'N'
  #   results.append(temp)
  
  tst = tokenizer1.texts_to_sequences(tweet_list)
  tst = pad_sequences(tst, maxlen=100)
  for pkl_filename in [filenames[17]]:
    res_arr = models3[pkl_filename].predict(tst)
    temp = {'F':0,'A':0,'N':0}
    for i in res_arr:
      temp[labels_stance[0]]+=i[0]
      temp[labels_stance[1]]+=i[1]
      temp[labels_stance[2]]+=i[2]
    # temp = {key:int(round(temp[key])) for key in temp.keys()}
    # fin = {key: (fin[key]+temp[key]) for key in temp.keys()}
    tot = sum(temp.values())
    temp = {key:temp[key]/tot for key in temp.keys()}
    if(temp['A'] > temp['F']):
      temp['res'] = 'A'
    elif(temp['A'] < temp['F']):
      temp['res'] = 'F'
    else:
      temp['res'] = 'N'
    results.append(temp['res'])

  # tst1 = tokenizer2.texts_to_sequences(tweet_list)
  # tst1 = pad_sequences(tst1, maxlen=100)
  # for pkl_filename in filenames[18:]:
  #   res_arr = models4[pkl_filename].predict(tst1)
  #   temp = {'F':0,'A':0,'N':0}
  #   for i in res_arr:
  #     temp[labels_stance[0]]+=i[0]
  #     temp[labels_stance[1]]+=i[1]
  #     temp[labels_stance[2]]+=i[2]
  #   # temp = {key:int(round(temp[key])) for key in temp.keys()}
  #   # fin = {key:fin[key]+temp[key] for key in temp.keys()}
  #   tot = sum(temp.values())
  #   temp = {key:temp[key]/tot for key in temp.keys()}
  #   if(temp['A'] > temp['F']):
  #     temp['res'] = 'A'
  #   elif(temp['A'] < temp['F']):
  #     temp['res'] = 'F'
  #   else:
  #     temp['res'] = 'N'
  #   results.append(temp)

  # df_res = pd.DataFrame(results)
  # display(results)
  
  # if( (results[3]=='F'and (results[0] == 'F' or results[2] == 'F')) or (results[2] == 'F')):
  if( results.count('F') >= 2 ):
    return 'F'
  elif( results.count('A') > 2 ):
    return 'A'
  else:
    return 'N'
  # return df_res['res'].values

# Get the propaganda and bias metric of user

### Characteristics of users who spread propaganda: 

(1) **sending high volumes of tweets over short periods of time** - check if time graph has peaks 

(2) **retweeting whilepublishing little original content** 

(3) **quickly retweeting**


### *(1) high tweet volume*

Avg person reads 200 words a minute with 60% comprehension

300-500 ms for regression + comprehension

Avg political tweet length + hindi tweet length 75
(280 characters allowed per tweet)

Avg char per word 5.1

Hence, Avg tweet words = 75/5 = 15

time per tweet = 15*(200/60) + 0.5 s = 50.5s

hence understandable tweets/minute = 1.18

Hence, if a person retweets 2 or more tweets per minute, they are participating in ill-informed propaganda


---



METRIC : N1/( total )

where N1 = tweets that are above 2 per minute in all categories


---



### *(2) less original content*

If retweet percentage much higher than both original and replies then - ill-informed propaganda (contributes more bias)

If retweet percentage very high but replies high too - informed propaganda (contributed less bias)


---


MERTIC : Retweet% - Original% - (Reply%)/2


---



### *(3) quick retweeting*

If time difference between original tweet time and retweet time is less than 50s it was mindlessly forwarded

---


MERTIC : N2/( total retweets )

where N2 = number of rts retweeted quickly

---
 combine with metric 1


In [ ]:
datetimeFormat = '%Y-%m-%d %H:%M:%S'
def getMetrics(df1,df2,df3,vals):
  # M1 = unique list of:
  # 1. anything in combo that was closer than 50 seconds
  # 2. any in retweets that are closer to original than 50 seconds
  # make unique list of these propaganda tweets
  # M2 = %(retweets) - %(originals) - (%replies)/2
  df_list = []
  prop_tweet = set()

  # for m1 - 2
  if(type(df1) != type(None)):
    for ind in df1.index: 
        date1 = str(df1['date'][ind])+' '+str(df1['time'][ind]) 
        date2 = str(df1['retweet_date'][ind])
        diff = datetime.datetime.strptime(date2, datetimeFormat) - datetime.datetime.strptime(date1, datetimeFormat)
        if(diff.total_seconds() < 51):
          prop_tweet.add(df1['id'][ind])
    df_list.append(df1)

  if(type(df2) != type(None)):
    df_list.append(df2)

  if(type(df3) != type(None)):
    df_list.append(df3)

  if(df_list == []):
    return 0,0

  # for m1 - 1 and stance
  df_comb = pd.concat(df_list)
  df_comb = df_comb[['date','time','id']]
  df_comb.reset_index(inplace=True, drop=True) 
  df_comb["datetime"] = df_comb['date'].astype(str)+' '+df_comb['time'].astype(str)
  final_df = df_comb.sort_values(by=['datetime'])

  # tweet_times = []
  for ind in range(len(final_df['datetime']) - 1):
    date1 = final_df["datetime"][final_df.index[ind]]
    date2 = final_df["datetime"][final_df.index[ind+1]]
    diff = datetime.datetime.strptime(date2, datetimeFormat) - datetime.datetime.strptime(date1, datetimeFormat)
    # tweet_times.append(diff.total_seconds())
    if(diff.total_seconds() < 51):
      prop_tweet.add(final_df["id"][ind])
    
  # percentage of ill-understood tweets
  m1 = len(prop_tweet)*100/len(df_comb['id'])
  # plt.plot(tweet_times)
  # plt.title(str(m1)+'% tweets propaganda - '+id)
  # plt.xlabel('difference in seconds')
  # plt.ylabel('number of tweets')
  # plt.savefig('/content/drive/My Drive/Graphs/graph7.png')
  # plt.show()
  m2 = (vals[0]/vals[3] - vals[1]/vals[3] - vals[2]/(vals[3]*2))*100
  # print(m2)
  if(m2 < 0):
    m2 = 0
  return m1,m2

# Load a user's details from drive,twint or cache

In [ ]:
all_dfs = {}

In [ ]:
#general get function
all_pths = [f for f in listdir('/content/drive/My Drive/TweetData')]
print(len(all_pths))
def getUser(user_id,offline=False):
  
  if(user_id in all_dfs.keys()):
    # print('present in array')
    return True
  
  if(not offline):
    df1, df2, df3, pth = getTweets(user_id,'caa OR nrc OR npr')
    
  if(offline or pth == "already there" ):
    pth = '/content/drive/My Drive/TweetData/tweets_'+str(user_id)
    try:
      df1 = pd.read_csv(pth+"/rts.csv", delimiter=',',names = col_names,skiprows=1)
    except:
      df1 = None
    try:
      df = pd.read_csv(pth+"/og_rep.csv", delimiter=',',names = col_names,skiprows=1)
      df2 = df[df['id'] == df['conversation_id']] #Original Tweets
      df2 = df2.reset_index(drop=True)
      df3 = df[df['id'] != df['conversation_id']] #Reply Tweets
      df3 = df3.reset_index(drop=True)
      stance = getPredictions(df['tweet'].values)
    except:
      # print(traceback.format_exc())
      stance = 'N'
      df2 = None
      df3 = None
    # print('loaded from drive')
  else:
    all_pths = [f for f in listdir('/content/drive/My Drive/TweetData')]
    # print('downloaded from twint')
  
  dfs = [df1, df2, df3]
  if(type(df1) !=  type(None)):
    c1 = df1.id.count()
  else:
    c1 = 0
  if(type(df2) !=  type(None)):
    c2 = df2.id.count()
  else:
    c2 = 0
  if(type(df3) !=  type(None)):
    c3 = df3.id.count()
  else:
    c3 = 0
  t = c1+c2+c3

  if(t!=0):
    m1,m2 = getMetrics(dfs[0],dfs[1],dfs[2],[c1,c2,c3,t])
    all_dfs[user_id] = {'dataframes':dfs,'values':[c1,c2,c3,t]}
    all_dfs[user_id]['metrics'] = [m1,m2]
    all_dfs[user_id]['stance'] = stance
    return True
  else:
    # print(user_id_i," error")
    return False


6847


## Populate the ego of most retweeted tweet

In [ ]:
influencial_users = list(set(df_most['user_id']))
for user_id_i in influencial_users:
  getUser(user_id_i)
  lst_rts = []
  for tw_id in all_dfs[user_id_i]['dataframes'][1]['id']:
    rts = api.retweets(str(tw_id),101)
    for rt in rts:
      lst_rts.append(rt._json['user']['id'])

  for user_id in lst_rts:
    getUser(user_id)


## Expansion of tweet graph

In [ ]:
lst_old = list(all_dfs.keys())
inferred_stance = {}
for i in lst_old:
  df1,df2,df3 = all_dfs[i]['dataframes']
  # add writers of retweets
  rt_stances = []
  if(type(df1) != type(None)):
    for new_id in list(set(df1['user_id'])):
      getUser(new_id)
        
  # add people replied to 
  if(type(df3) != type(None)):
    for ind in df3.index:
      try:
        new_id =  ast.literal_eval(df3['reply_to'][ind])[1]['user_id']
        getUser(new_id)
      except:
        ;

### Load all users whose data is present in drive

In [ ]:
# load all from memory
all_pths = [f for f in listdir('/content/drive/My Drive/TweetData')]
for pth1 in all_pths: 
  user_id_i = pth1.split("_",1)[1]
  print(getUser(user_id_i))

True
translator failed
True
translator failed
True
True
translator failed
True
translator failed
True
translator failed
True
translator failed
True
translator failed
True
translator failed
True
translator failed
True
True
translator failed
True
translator failed
True
translator failed
True
translator failed
True
True
translator failed
True
True
translator failed
True
True
True
translator failed
True
translator failed
True
translator failed
True
translator failed
True
translator failed
True
translator failed
True
True
translator failed
True
translator failed
True
translator failed
True
translator failed
True
True
translator failed
True
translator failed
True
translator failed
True
True
translator failed
True
translator failed
True
translator failed
True
translator failed
True
translator failed
True
translator failed
True
True
translator failed
True
translator failed
True
True
False
False
False
False
False
False
False
False
translator failed
True
True
True
True
translator failed
True
Tru

# Get inferred stances

Higher reliability on stance predicted for users with lot of original content. Hence, more confidence in tweets that are inferred then on model predictions

In [ ]:
# Check stances with retweets
lst_old = list(all_dfs.keys())
inferred_stance = {}
for i in lst_old:
  df1,df2,df3 = all_dfs[i]['dataframes']
  # add writers of retweets
  rt_stances = []
  if(type(df1) != type(None)):
    for new_id in list(set(df1['user_id'])):
      if(getUser(new_id,True)):
        rt_stances.append(all_dfs[new_id]['stance'])
  if(rt_stances == []):
    continue
  if(rt_stances.count('F') > rt_stances.count('A')):
    inferred_stance[i] = 'F'
  elif(rt_stances.count('F') < rt_stances.count('A')):
    inferred_stance[i] = 'A'
  else:
    inferred_stance[i] = 'N'

In [ ]:
wr = []
for i in inferred_stance:
  if(inferred_stance[i] != all_dfs[i]['stance'] ):
    wr.append(i)
print(len(wr))
print(len(inferred_stance))


2195
3070


In [ ]:
p_list = []
for i in all_dfs:
  temp = {}
  temp['id'] = i
  temp['ret'] = all_dfs[i]['values'][0]
  temp['og'] = all_dfs[i]['values'][1]
  temp['rep'] = all_dfs[i]['values'][2]
  temp['tot'] = all_dfs[i]['values'][3]
  temp['m1'] = all_dfs[i]['metrics'][0]
  temp['m2'] = all_dfs[i]['metrics'][1]
  if(i in inferred_stance.keys()):
    temp['stance'] = inferred_stance[i]
  else:
    temp['stance'] = all_dfs[i]['stance']
  p_list.append(temp)

df_metrics1 = pd.DataFrame(p_list) 
df_metrics1.to_csv('/content/drive/My Drive/metrics_inferred1.csv')

### Testing stance

In [ ]:
interest_u = ['85657578','1153045459','17537467','18839785','889481829966782466','3171712086','1215480693200805888']

In [ ]:
for u_id in list(set(interest_u)):
  if(getUser(u_id)):
    s_res = all_dfs[u_id]['stance']
    display(set(all_dfs[u_id]['dataframes'][1]['username']))
    print(s_res)
  else:
    print(u_id,' error')

{'incindia'}

A


{'tarekfatah'}

F


{'shrutiv_vyas'}

...

F


{'rahulgandhi'}

A


{'narendramodi'}

F


### Present the metrics according to stance

In [ ]:
def present_metrics(df_metrics,topic):
  warnings.filterwarnings("ignore")
  print(topic)
  
  print('\nFOR')
  tf = len(df_metrics[df_metrics['stance'] == 'F'])
  print('total users',tf)
  pf = len(df_metrics[df_metrics['stance'] == 'F'][df_metrics['m1'] > 0])
  print('total propaganda users',pf)
  print('propaganda user % ',pf*100/tf)
  print('mean propaganda',df_metrics[df_metrics['stance'] == 'F']['m1'].mean())
  print('total unoriginal users',len(df_metrics[df_metrics['stance'] == 'F'][df_metrics['m2'] > 0]))
  print('mean unoriginality',df_metrics[df_metrics['stance'] == 'F']['m2'].mean())

  print('\nAGAINST')
  tf = len(df_metrics[df_metrics['stance'] == 'A'])
  print('total users',tf)
  pf = len(df_metrics[df_metrics['stance'] == 'A'][df_metrics['m1'] > 0])
  print('total propaganda users',pf)
  print('propaganda user % ',pf*100/tf)
  print('mean propaganda',df_metrics[df_metrics['stance'] == 'A']['m1'].mean())
  print('total unoriginal users',len(df_metrics[df_metrics['stance'] == 'A'][df_metrics['m2'] > 0]))
  print('mean unoriginality',df_metrics[df_metrics['stance'] == 'A']['m2'].mean())

  print('\nNONE')
  tf = len(df_metrics[df_metrics['stance'] == 'N'])
  print('total users',tf)
  pf = len(df_metrics[df_metrics['stance'] == 'N'][df_metrics['m1'] > 0])
  print('total propaganda users',pf)
  print('propaganda user % ',pf*100/tf)
  print('mean propaganda',df_metrics[df_metrics['stance'] == 'N']['m1'].mean())
  print('total unoriginal users',len(df_metrics[df_metrics['stance'] == 'N'][df_metrics['m2'] > 0]))
  print('mean unoriginality',df_metrics[df_metrics['stance'] == 'N']['m2'].mean())
  display(HTML("<hr/>"))

In [ ]:
import warnings
# present_metrics(df_metrics,'Using model stance')
present_metrics(df_metrics1,'Using stance from model + inferring from graph')
df_metrics2 = df_metrics1[df_metrics1['tot'] > 15]
present_metrics(df_metrics2,'Filtered users')

Using stance from model + inferring from graph

FOR
total users 2051
total propaganda users 578
propaganda user %  28.18137493905412
mean propaganda 2.6192086247181017
total unoriginal users 505
mean unoriginality 17.36165829563697

AGAINST
total users 2642
total propaganda users 591
propaganda user %  22.369417108251326
mean propaganda 1.9171069356631656
total unoriginal users 867
mean unoriginality 23.573975872582228

NONE
total users 2717
total propaganda users 338
propaganda user %  12.440191387559809
mean propaganda 1.8220227919118392
total unoriginal users 492
mean unoriginality 14.263956791415103


Filtered users

FOR
total users 956
total propaganda users 487
propaganda user %  50.94142259414226
mean propaganda 3.793881034780552
total unoriginal users 73
mean unoriginality 3.5722842162070836

AGAINST
total users 1173
total propaganda users 516
propaganda user %  43.989769820971865
mean propaganda 3.1930287846959207
total unoriginal users 133
mean unoriginality 6.097662311786741

NONE
total users 430
total propaganda users 196
propaganda user %  45.58139534883721
mean propaganda 4.116339669103711
total unoriginal users 21
mean unoriginality 1.8950689377532417


Hence, even though the number of people with stance of 'FOR' and 'AGAINST' are different, the higher propaganda on the 'FOR' side

### Observation at 4000 users 

In [ ]:
df_metrics['m1'].mean()

1.9446539643335663

In [ ]:
print('percentage of all current users that participate in propaganda ')
len(df_metrics[df_metrics['m1']>0]['id'])*100/len(df_metrics['id'])

 percentage of all current users that participate in propaganda 


16.10922604621102

### Checking for bots

In [ ]:
#x-rapidapi-key": "930c3333aamsh6abe1fa37394f1dp1d3792jsne8c0dd43343f
import botometer
rapidapi_key = "930c3333aamsh6abe1fa37394f1dp1d3792jsne8c0dd43343f" # now it's called rapidapi key
twitter_app_auth = {
    'consumer_key': 'aki0CkgST6uEPvQQAbApbgSjU',
    'consumer_secret': 'GV63N7Gow7jM8MS7n1P7AbBg244kpjwp7jlfH8hoiWCkvgQ6CT',
    'access_token': '2199543638-X9myvLXlyDt7AtUkB7TEcSOYYdqPAtPMjcAFJ5J',
    'access_token_secret': '7B3LTEQUwDXh3zrDvOkBdi0pHXAxDzHInE64yDXoDw6yV',
  }
bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)

def getBotScore(usr_id):
  result = bom.check_account(int(usr_id))
  return max(result['cap']['english'],result['cap']['universal'])

In [ ]:
list1 = list(df_metrics1.id.values)
ind = 0
while(ind < len(list1)):
  bot_res = []
  for i in list1[ind:]:
        try:
          val = getBotScore(i)
          bot_res.append({"id":i,"cap":val})
        except:
          print(sys.exc_info()[0])
          print('error at '+str(i))
          last_val = i
          break
  df_bot = pd.DataFrame(bot_res) 
  df_bot.to_csv('/content/drive/My Drive/bot_data/bot_file'+str(ind)+'.csv')
  ind = list1.index(last_val)+1

<class 'tweepy.error.TweepError'>
error at 942748218424623105
<class 'tweepy.error.TweepError'>
error at 714265230721699840


In [ ]:
all_pthsb = [f for f in listdir('/content/drive/My Drive/bot_data')]
dfsb = []
for fb in all_pthsb:
  dfb = pd.read_csv('/content/drive/My Drive/bot_data/'+fb, delimiter=',',names = ['id','cap'],skiprows=1)
  dfsb.append(dfb)

In [ ]:
df_caps = pd.concat(dfsb,ignore_index=True)

In [ ]:
statistics.mean(df_caps['cap'].values)

0.05264535721879503

In [ ]:
stncs = []
for i in df_caps['id'].values:
  stncs.append(df_metrics1[df_metrics1['id'] == i]['stance'].values[0])
len(stncs)

285

In [ ]:
df_caps['stance'] = stncs

In [ ]:
print('A',statistics.mean(df_caps[df_caps['stance'] == 'A']['cap'].values))
print('F',statistics.mean(df_caps[df_caps['stance'] == 'F']['cap'].values))
print('N',statistics.mean(df_caps[df_caps['stance'] == 'N']['cap'].values))

A 0.04660138435267751
F 0.05989869426898036
N 0.05255728219975074


In [ ]:
print('A',df_caps[df_caps['stance'] == 'A']['id'].count())
print('F',df_caps[df_caps['stance'] == 'F']['id'].count())
print('N',df_caps[df_caps['stance'] == 'N']['id'].count())

A 87
F 74
N 124


In [ ]:
print(df_caps[df_caps['cap'] > 0.50])

                      id       cap stance
9    1200420801876500481  0.641146      N
69   1102905529980968961  0.938706      N
226  1145902517462564870  0.511452      F
231   794792628638322689  0.511452      N


# Visualizations

In [ ]:
df_metrics1 = pd.read_csv('/content/drive/My Drive/metrics_inferred1.csv', delimiter=',',names = ['id','ret','og','rep','tot','m1','m2','stance'],skiprows=1)

In [ ]:
def getColor(id):
  if(id in df_metrics1['id'].values):
    if(df_metrics1[df_metrics1['id'] == id]['stance'].values == ['F']):
        return 'green'
    elif(df_metrics1[df_metrics1['id'] == id]['stance'].values == ['A']):
        return 'red' 
    else:
      return 'white'
  else:
    if(all_dfs[id]['stance'] == 'F'):
        return 'green'
    elif(all_dfs[id]['stance'] == ['A']):
        return 'red' 
    else:
      return 'white'
    
# pos=nx.spring_layout(G)
# plt.figure(1,figsize=(12,12)) 
# nx.draw(G,node_color=color_map,node_size=2)
# plt.savefig('/content/drive/My Drive/Graphs/graph10.png')
 

In [ ]:
G = nx.DiGraph()
nf = 0
for i in all_dfs:
  df1,df2,df3 = all_dfs[i]['dataframes']
  if(type(df1) != type(None)):
    if(i not in G.nodes):
        G.add_node(i,color=getColor(i))
    for new_id in list(set(df1['user_id'])):
      if(new_id in all_dfs.keys()):
        if(new_id not in G.nodes):
          G.add_node(new_id,color=getColor(new_id))
        G.add_edge(new_id, i, weight=2)
      else:
        nf+=1

  if(type(df3) != type(None)):
    if(i not in G.nodes):
        G.add_node(i,color=getColor(i))
    for ind in df3.index:
      try:
        new_id =  ast.literal_eval(df3['reply_to'][ind])[1]['user_id']
        if( new_id in all_dfs.keys()):
          if(new_id not in G.nodes):
            G.add_node(new_id,color=getColor(new_id))
          G.add_edge(i, new_id, weight=1)
        else:
          nf+=1
      except:
        # reply to self
        continue

nx.write_gexf(G, "test.gexf")

In [ ]:
def plot_activity(df1,title,kind='line'):
  df1["date"] = df1["date"].astype("datetime64")
  df1["time"] = df1["time"].astype("datetime64")
  df1 = df1[["time","date","tweet"]]
  return df1.groupby([df1["time"].dt.minute,df1["time"].dt.hour,df1["date"].dt.month, df1["date"].dt.date])['tweet'].count().plot(kind=kind,figsize=(15,15),x ='time',y='number of tweets',title=title)

In [ ]:
df1,df2,df3 = all_dfs['1108723895207628800']['dataframes']
ax = plot_activity(df3.copy(deep=True),'reply content by minutes - 1108723895207628800 ')
# ax.figure.savefig('/content/drive/My Drive/Graphs/graph5.png')
ax